In [ ]:
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")

from warnings import filterwarnings

filterwarnings("ignore")

import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc("font", **{"size": 22})

from investment_lab.option_selection import select_options
from investment_lab.metrics.volatility import rolling_realized_volatility
from investment_lab.metrics.util import levels_to_returns
from investment_lab.data.option_db import OptionLoader, extract_spot_from_options
from investment_lab.data.rates_db import USRatesLoader
from investment_lab.pricing.black_scholes import black_scholes_price, black_scholes_greeks
from investment_lab.rates import compute_forward
from investment_lab.metrics.distance import mse


In [ ]:
df_options = OptionLoader.load_data(datetime(2022, 1, 4), datetime(2022, 1, 4), process_kwargs={"ticker":"SPY"},)
df_options.head()

In [ ]:
df_spot = extract_spot_from_options(df_options)
df_spot.head()

In [ ]:
df_rates = USRatesLoader.load_data(datetime(2022, 1, 4), datetime(2022, 1, 4))
df_rates.head()

In [ ]:
df_options = compute_forward(df_options=df_options, df_rates=df_rates)
df_options.head()